# 01 — From C to C++: The Toolchain 🛠️

## 🎯 Goal
As a C programmer, you know how code transforms from text to binary (Preprocessing → Compilation → Assembly → Linking). This does not change in C++. 

However, the tools and the file organization do change to accommodate C++'s complexity (Templates, Name Mangling, Standard Library).

In this notebook, we will cover:
1. g++ vs gcc
2. File Extensions (.hpp vs .h)
3. The Header/Source Split in C++
4. Modern Build Systems (Why CMake?)

---

## 1. The Compiler: g++

In C, you use gcc. In C++, you generally use g++ (or clang++).

### ❓ Why can't I just use gcc for C++ files?
You actually can, but you shouldn't. 

* gcc treats files ending in .cpp as C++ files (correctly).
* HOWEVER, gcc does not automatically link the C++ Standard Library (libstdc++ or libc++).

If you try to compile C++ object code with gcc, you will likely get undefined reference errors for symbols like std::cout or std::string.

### The Command Line
Just like C, you can compile single files. Note the -std=c++17 flag. C++ evolves faster than C; so always specify the version.

- The C Way

```bash
gcc main.c -std=c99 -o app
```
- The C++ Way

```bash
g++ main.cpp -std=c++17 -Wall -Wextra -o app
```

### 🧪 Interactive Check: The "Hello World" Difference

In C, you rely on stdio.h and printf. In C++, we use the Input/Output Stream library (iostream).

Notice:
1. No .h in the standard header include (\<iostream\>, not <iostream.h>).
2. std:: namespace prefix.
3. << operator for output.

In [5]:
// Run this cell (Shift + Enter)
#include <iostream>

{
    int year = 2023;
    // Type-safe printing. No %d formatting codes required.
    std::cout << "Hello from C++! The year is " << year << std::endl;
    return 0;
}

Hello from C++! The year is 2023


0

---

## 2. File Extensions & Headers (.hpp)

In C, the convention is strict: .c for source, .h for headers.

In C++, it's a bit looser, but here is the Professional Standard:

| Type | Extension | Note |
| :--- | :--- | :--- |
| Source | .cpp | Avoid .C or .cxx if possible for consistency. |
| Header | .hpp | Distinguishes C++ headers (which may contain templates/classes) from C headers. |

### Namespaces (used in our example)
In `calculator_project/include/simple_math.hpp` we wrap functions inside `namespace MathUtils { ... }`.
* Namespaces group related symbols and prevent name collisions (e.g., two libraries both defining `add`).
* Access members with the scope operator: `MathUtils::add(2, 3)`.
* Avoid `using namespace` in headers; it leaks names into every file that includes them.

### ⚠️ The "extern C" Issue
C++ compilers "mangle" function names to support Overloading (having two functions with the same name but different arguments).

When you call a function that was compiled by a C compiler (no mangling), the C++ linker will not find the symbol unless you keep the plain C name. Wrap declarations from pure-C headers like this:

```cpp
#ifdef __cplusplus
extern "C" {
#endif

void my_c_function();

#ifdef __cplusplus
}
#endif
```

Why?
* `extern "C"` tells the C++ compiler: do not mangle these names, look for them exactly as a C object file exports them.
* You only need this wrapper when including a C header inside C++ code. If everything is compiled as C++ (or the header is only used by C code), you can skip it.

---


## 3. The Build System: Entering CMake

Coming from C, you likely love (or hate) Makefiles.

In C++, dependencies are more complex. Linking OpenMP, CUDA, or Boost in a Makefile requires complex flag management. CMake solves this by abstracting the build process.

CMake is not a compiler. It is a Build System Generator. 
It reads a CMakeLists.txt file and generates a Makefile for you.

### Structure of a Modern C++ Project

We typically organize files like this (see the `calculator_project/` folder in this directory):

```text
calculator_project/
├── CMakeLists.txt       <-- The Recipe
├── include/
│   └── simple_math.hpp  <-- The Declaration (.hpp)
├── src/
│   ├── simple_math.cpp  <-- The Implementation (.cpp)
│   └── main.cpp         <-- The Entry Point
└── build/               <-- We compile here (keeps source clean)
```

### The CMakeLists.txt Recipe

Here is how we translate a Makefile logic to CMake:

```cmake
cmake_minimum_required(VERSION 3.10)
project(MyCalc)

# 1. Set Version
set(CMAKE_CXX_STANDARD 17)

# 2. Include Directories (Where are the .hpp files?)
# We do not add -I flags manually; CMake tracks the include path.
include_directories(include)

# 3. Create the Executable
# "Make an app called "calculator" using these source files"
add_executable(calculator 
    src/main.cpp 
    src/simple_math.cpp
)
```

### How to compile calculator_project

By running the commands bellow you can use cmake to generate a makefile, and them run the make command to compile our calculator project.

```bash
cd 01_From_C_to_Cpp/calculator_project
mkdir -p build/manual && cd build/manual
cmake ../..
make
./calculator
```

### 📦 CMakePresets.json: Repeatable Configure/Build

To make things simple we can also have a `CMakePresets.json` that can be used to automate paths, and configurations.

For this module we keep a minimal preset at `01_From_C_to_Cpp/calculator_project/CMakePresets.json` with a single `default` (Debug) configure/build preset.

* **Configure preset**: generator (Unix Makefiles), Debug flags, exports `compile_commands.json`, writes to `build/default`.
* **Build preset**: reuses that configure preset so artifacts always land in the same folder.

Try it from the project folder:
```bash
cd 01_From_C_to_Cpp/calculator_project
cmake --list-presets
cmake --preset default   # configure and generate Makefiles
make -C build/default    # build using the generated Makefile
```

Benefit: zero guessing about flags or build folders, and the CMake Tools extension reads it automatically.


### 🛠️ VS Code + CMake Tools Extension

We can also use an Vscode extention to automate that process. Install **CMake Tools** in VS Code to drive CMake without leaving the editor. It understands `CMakePresets.json` and uses our `.vscode/settings.json` defaults.

Workflow:
1) Command Palette → `CMake: Select Configure Preset` → choose `default`.
2) `CMake: Build` uses that preset (generator + flags) and writes to `build/default`.
3) In `Project Outline` section inside `Cmake` extension you can select build all projects

If you switch modules, update `.vscode/settings.json` so VS Code knows which `CMakeLists.txt` to use. For this chapter we point it to `01_From_C_to_Cpp/calculator_project` so hitting **Build** runs CMake there.


### 📝 Exercise: Compile the Project

Two easy ways to build `calculator_project`:

1) **Terminal (make first)**:
```bash
cd 01_From_C_to_Cpp/calculator_project
cmake --preset default   # configure + generate Makefiles
make -C build/default    # build
./build/default/calculator
```

2) **VS Code + CMake Tools** (recommended): Select the `default` preset and hit **Build** (CMake Tools will call `make` for you).